In [1]:
from sqlalchemy import create_engine, inspect, select, Table, MetaData
from typing import List
import pandas as pd
import datetime
import numpy as np
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['TRANSFORMERS_CACHE'] = 'cache'
from tabulate import tabulate

In [2]:
username = os.environ["username"]
password = os.environ["password"]
hostname = os.environ["hostname"]
port = os.environ["port"]

In [8]:
credentials = f"postgresql://{username}:{password}@{hostname}:{port}/aact"
engine = create_engine(credentials)

# create metadata object and reflect all tables
metadata = MetaData()
metadata.reflect(bind=engine)

sql_command = \
"""
SELECT
    studies.nct_id,
    MAX(studies.brief_title) AS brief_title,
    MAX(studies.official_title) AS official_title,
    STRING_AGG(DISTINCT brief_summaries.description, ' ') AS brief_summaries,
    STRING_AGG(DISTINCT detailed_descriptions.description, ' ') AS detailed_descriptions,
    MAX(eligibilities.criteria) AS criteria, 
    MAX(eligibilities.gender) AS gender, 
    MAX(eligibilities.minimum_age) AS minimum_age, 
    MAX(eligibilities.maximum_age) AS maximum_age, 
    MAX(facilities.name) AS facilities, 
    MAX(facilities.city) AS city, 
    MAX(facilities.state) AS state, 
    MAX(facilities.zip) AS zip, 
    MAX(facilities.country) AS country, 
    MAX(studies.study_type) AS study_type
FROM 
    ctgov.studies
INNER JOIN ctgov.brief_summaries ON brief_summaries.nct_id = studies.nct_id 
INNER JOIN ctgov.detailed_descriptions ON detailed_descriptions.nct_id = studies.nct_id 
INNER JOIN ctgov.eligibilities ON eligibilities.nct_id = studies.nct_id 
INNER JOIN ctgov.facilities ON facilities.nct_id = studies.nct_id 
GROUP BY studies.nct_id
ORDER BY studies.nct_id ASC;
"""

assert sql_command is not None
df = pd.read_sql_query(
    sql=sql_command,
    con=engine,
    # params={
    #    # "prompt": f"%{self.prompt}%",
    # }  # type: ignore
)

# Set 'NCTId' as index and remove the 'NCTId' column
df.set_index(df['nct_id'], inplace=True)
df.drop(columns=['nct_id'], inplace=True)

# Convert lists to appropriate data types
for column in df.columns:
    # Remove double quotes from the DataFrame
    df[column] = df[column].apply(lambda x: x.replace('"', '') if isinstance(x, str) else x)
    df[column] = pd.to_datetime(df[column], errors='coerce') if 'date' in column else df[column]

# Close the connection
engine.dispose()

df.to_csv("ctgov.csv")
df

,brief_title,official_title,brief_summaries,detailed_descriptions,criteria,gender,minimum_age,maximum_age,facilities,city,state,zip,country,study_type
nct_id,,,,,,,,,,,,,,
NCT00000102,Congenital Adrenal Hyperplasia: Calcium Channe...,None,This study will test the ability of extended r...,This protocol is designed to assess both acute...,Inclusion Criteria:\n\ndiagnosed with Congenit...,All,14 Years,35 Years,Medical University of South Carolina,Charleston,South Carolina,None,United States,Interventional
NCT00000105,Vaccination With Tetanus and KLH to Assess Imm...,Vaccination With Tetanus Toxoid and Keyhole Li...,The purpose of this study is to learn how the ...,Patients will receive each vaccine once only c...,Inclusion Criteria:\n\nPatients must have a di...,All,18 Years,N/A,"Division of Hematology, Oncology, and Transpla...",Minneapolis,Minnesota,55455,United States,Observational
NCT00000113,Correction of Myopia Evaluation Trial (COMET),Correction of Myopia Evaluation Trial (COMET),To evaluate whether progressive addition lense...,Myopia (nearsightedness) is an important publi...,Children between the ages of 6 and 12 years wi...,All,6 Years,12 Years,"University of Houston, College of Optometry",Philadelphia,Texas,77204-6052,United States,Interventional
NCT00000116,Randomized Trial of DHA for Retinitis Pigmento...,Clinical Trial of Docosahexaenoic Acid (DHA) i...,The purpose of this trial is to determine whet...,Retinitis pigmentosa (RP) is a group of inheri...,Inclusion Criteria:\n\nOcular Criteria:\n\nRet...,All,18 Years,55 Years,Berman-Gund Laboratory for the Study of Retina...,Boston,Massachusetts,02114,United States,Interventional
NCT00000117,Intravenous Immunoglobulin Therapy in Optic Ne...,None,To determine whether high-dose intravenous imm...,Optic neuritis is the leading cause of transie...,"To be eligible, patients must have a history o...",All,N/A,50 Years,"Mayo Clinic, Department of Neurology",Rochester,Minnesota,None,United States,Interventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCT05808725,Amlodipine Freeze-Dried Powder for Oral Soluti...,Relative Bioavailability Study of Amlodipine P...,The goal of this clinical study is to assess t...,"A total of 24 healthy, adult, male and female ...","Inclusion Criteria:\n\nAdult, Healthy",All,18 Years,N/A,Accutest Research Laboratories (I) Pvt. Ltd.,Vadodara,Gujarat,390016,India,Interventional
NCT05808751,Clinical Cardiac Rehabilitation Registry Study,Clinical Cardiac Rehabilitation Registry Study,Establish an electronic cardiac rehabilitation...,Establish an electronic cardiac rehabilitation...,Inclusion Criteria:\n\n18 years of age and old...,All,18 Years,N/A,"Fuwai Hospital, National Center for Cardiovasc...",Beijing,None,None,China,Observational [Patient Registry]
NCT05808777,Validation of the ICH Score for the Prediction...,Validation of the ICH Score for the Prediction...,The goal of this observational study is to ana...,The ICH Score had been utilized to predict the...,Inclusion Criteria:\n\nAged 20 years or older\...,All,20 Years,N/A,National Taiwan University Hospital,Taipei,None,100,Taiwan,Observational


In [4]:
credentials = f"postgresql://{username}:{password}@{hostname}:{port}/mimiciii"
engine = create_engine(credentials)

# create metadata object and reflect all tables
metadata = MetaData()
metadata.reflect(bind=engine)

sql_command = \
"""
SELECT
    noteevents.subject_id,
    MAX(noteevents.hadm_id) AS hadm_id,
    MAX(noteevents.category) AS category,
    MAX(noteevents.description) AS description,
    STRING_AGG(DISTINCT noteevents.text, ' ') AS text
FROM 
    mimic.noteevents
GROUP BY noteevents.subject_id
ORDER BY noteevents.subject_id ASC
"""

assert sql_command is not None
df = pd.read_sql_query(
    sql=sql_command,
    con=engine,
)

# Set 'NCTId' as index and remove the 'NCTId' column
df.set_index(df['subject_id'], inplace=True)
df.drop(columns=['subject_id'], inplace=True)

# Convert lists to appropriate data types
for column in df.columns:
    # Remove double quotes from the DataFrame
    df[column] = df[column].apply(lambda x: x.replace(
        '"', '') if isinstance(x, str) else x)
    df[column] = pd.to_datetime(
        df[column], errors='coerce') if 'date' in column else df[column]

# Close the connection
engine.dispose()

df.to_csv("mimiciii.csv")
df

,hadm_id,category,description,text
subject_id,,,,
2,163353.0,Nursing/other,Report,Neonatology Attending Triage Note\n\nBaby [**N...
3,145834.0,Radiology,VIDEO OROPHARYNGEAL SWALLOW,\n\n\n [**2101-10-11**] 9:06 PM\n CHEST (PRE-O...
4,185777.0,Radiology,"US ABD LIMIT, SINGLE ORGAN","[**2187**]: Pt c/o cough, non-productive, dry...."
5,178980.0,Nursing/other,Report,NICU Nursing Septic Workup Note\n[**Name8 (MD)...
6,107064.0,Radiology,R UNILAT LOWER EXT VEINS RIGHT,[**2175-5-25**] 10:52 AM\n CHEST (PRE-OP PA & ...
...,...,...,...,...
99985,176670.0,Social Work,Social Work Progress Note,53 yo male pt with h/o ? PNA transffered from ...
99991,151118.0,Rehab Services,Report,[**2184-12-27**] 11:35 AM\n CHEST (PA & LAT) ...
99992,197084.0,Radiology,R UNILAT UP EXT VEINS US RIGHT,[**2144-7-10**] 8:04 AM\n CHEST PORT. LINE PLA...
